In [1]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skan import skeleton_to_csgraph
from skimage import io, morphology
from plantcv import plantcv as pcv
import math
from skimage.morphology import skeletonize
from skimage import data
from skimage.util import invert
from scipy import spatial
from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table, EllipseModel
from skimage.transform import rotate
import pandas as pd 
import time
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import random
from statistics import mean 
import gc 

/home/dhruv/anaconda3/lib/python3.7/site-packages/skan/csr.py:21: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @numba.jitclass(csr_spec)


In [2]:
def invertBinaryImage(im):
    ## Flips the image pixel values(0 and 255) and returns the inverted image
    inv = np.zeros(im.shape)
    inv[im==0] = 255
    return inv

def show_scalled_img(img_arr, scalePercent=100):
    ## shows the image at a scale percent of the origianl size.   
    scale_percent = scalePercent

    #calculate the 50 percent of original dimensions
    width = int(img_arr.shape[1] * scale_percent / 100)
    height = int(img_arr.shape[0] * scale_percent / 100)

    # resize image
    output = cv2.resize(img_arr, (width, height))

    fig = plt.figure(figsize = (50,50))
    plt.imshow(output, cmap='gray', vmin=0, vmax=255)
    plt.show()

## Connected Components: 
def DFSUtil(temp, v, visited, numEllipse, adjacencyMat):
    N = numEllipse
    # Mark the current vertex as visited
    visited[v] = 1
    
    # Store the vertex to list 
    temp.append(v)
    
    # Repeat for all vertices adjacent
    for i in range(N):
        if adjacencyMat[v][i] == 1:
            if visited[i] == 0:
                # Update the list
                temp = DFSUtil(temp, i, visited, N, adjacencyMat)
    #print(temp)
    return temp

In [3]:
############### OPERATIONS USED IN GRATE ############################3

def BlurThresh(img, num_interation, dspace_pix, frac):
    input = img
    Blur_k_size = int(dspace_pix*frac)
    
    if Blur_k_size%2 != 1:
        Blur_k_size = Blur_k_size+1

    blur = input
    for i in range(num_interation):
        blur = cv2.GaussianBlur(blur,(Blur_k_size,Blur_k_size),0)
    _,thresh = cv2.threshold(blur,0, 255,  cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return thresh

# Closing: Removing black spots from white regions. Basically Dilation followed by Erosion.
def Closing(img, k_size):
    input = img
    kernel = np.ones((k_size,k_size),np.uint8)
    output = cv2.morphologyEx(input, cv2.MORPH_CLOSE, kernel)
    return output

## Opening: Removing white spots from black regions. Basically Erosion followed by Dilation.
def Opening(img, k_size):
    input = img
    kernel = np.ones((k_size,k_size),np.uint8)
    output = cv2.morphologyEx(input, cv2.MORPH_OPEN, kernel)
    return output 

## Skeletonize Image
def Skeletonize(img): 
    input = img 
    image = invert(input/np.max(input))
    skeleton = skeletonize(image)
    output = (skeleton/np.max(skeleton))*255 
    return output

## Finds branching points for images with black background and white lines receive a degree matrix
def BreakBraches(img):
    input = np.copy(img)
    _, _, degrees = skeleton_to_csgraph(input)
    # consider all values larger than two as intersection
    intersection_matrix = degrees > 2
    input[intersection_matrix==True] = 0
    return input

## Skeleton Segmentation
def SkeletonSegmentation(img):
    input = np.copy(img)
    input = input.astype('uint8')
    # Set global debug behavior to None (default), "print" (to file), or "plot" (Jupyter Notebooks or X11)
    pcv.params.debug = "None"
    pcv.params.line_thickness = 2
    segmentedImg, obj = pcv.morphology.segment_skeleton(skel_img=input)
    
    return segmentedImg, obj

## Filtering out small backbones using the pixThresh variable and breaking them into uniform size.
def Filtered_Uniform_BB(img, obj_list, pixelThreshold, ellipseSize):
    bb = np.zeros(img.shape)
    residualFrac = 0.3
    minResidual = int(residualFrac*ellipseSize)
    
    for i in range(len(obj_list)):
        BoneLen = len(obj_list[i]) 
        if BoneLen > pixelThreshold: #and BoneLen>minResidual:
            #count = 0
            for ind,value in enumerate(obj_list[i]):
                #count += 1
                if (ind+1)%ellipseSize == 0:
                    bb[value[0][1],value[0][0]] = 0
#                     if (BoneLen - (ind+1)) < minResidual:
#                         break
                else:
                    bb[value[0][1],value[0][0]] = 255
        BoneLen = 0
    return bb

# ## Filtering out small backbones using the pixThresh variable and breaking them into uniform size.
# def Remove_small_BB(img, obj_list, pixelThreshold):
#     bb = np.zeros(img.shape)
#     for i in range(len(obj_list)):
#         if len(obj_list[i]) > pixelThreshold:
#             for ind in obj_list[i]:
#                 bb[ind[0][1],ind[0][0]] = 255
#     return bb

## Removing unnecessary dimension from the filteredPoly and storing it in restructuredFP
def RemovingDim(lis):
    temp = []
    for i in range(len(lis)):
        tp = np.zeros((len(lis[i]), 2))
        for j,val in enumerate(lis[i]):
            tp[j][0] = val[0][0]
            tp[j][1] = val[0][1]
        temp.append(tp)
    return temp


################ SCIPY BASED ELLIPSE CONSTRUCTION FUNCTION ###############################
def EllipseConstruction(img, aspectRatio):
    input = np.copy(img)
    label_img = label(input)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length'))
    props = pd.DataFrame(props)
    props['orientation'] = -90-props['orientation']*180/np.pi
    props['major_axis_length'] = props['major_axis_length']/2
    props['minor_axis_length'] = props['minor_axis_length']/2

    bb_props = pd.DataFrame()
    bb_props = props
    
    for ind in range(props.shape[0]):
        if props['minor_axis_length'][ind] < 1:
            bb_props = bb_props.drop([ind])   
        elif props['major_axis_length'][ind]/props['minor_axis_length'][ind] > aspectRatio:
            
            ellip_temp_img = cv2.ellipse(input, (int(props['centroid-1'][ind]), int(props['centroid-0'][ind])), \
                                         (int(props['major_axis_length'][ind]),int(props['minor_axis_length'][ind])),\
                                         int(props['orientation'][ind]), 0.0, 360.0, (255, 0, 0), 2);
        else: 
            bb_props = bb_props.drop([ind])
    
    bb_props_np = bb_props.to_numpy()
    return ellip_temp_img, bb_props_np

## Creating Adjacency Matrix based on distance between centroid and the orientation angle
def AdjacencyMat(img, bb_props, distanceThresh, thetaThresh):
    
    bb_props_np = bb_props
    N = len(bb_props_np)
    A_Mat = np.zeros((N,N))
    for i in range(N):
        for j in range(N): ## Change j to i+1 to N
            if i == j:
                continue
            else:
                #l2norm = np.linalg.norm([bb_props_np[i][0]-bb_props_np[j][0],bb_props_np[i][1]-bb_props_np[j][1]])
                pts1 = majorAxisPoints(bb_props_np[j])
                pts2 = majorAxisPoints(bb_props_np[i])
                
                l2norm = minDist(pts1, pts2)
                angleDiff = abs(bb_props_np[i][2]-bb_props_np[j][2])
                if l2norm < distanceThresh and angleDiff < thetaThresh:
                    A_Mat[i][j] = 1
                    A_Mat[j][i] = 1
    
    # Plotting the adjacent ellipse.
    A_Mat_img = np.copy(img)
    for i in range(N):
        for j in range(N):
            if A_Mat[i][j] == 1:
                poly = bb_props_np[i]
                A_Mat_img = cv2.ellipse(A_Mat_img, (int(poly[1]), int(poly[0])), (int(poly[3]), int(poly[4])), poly[2], 0.0, 360.0, (255, 0, 0), 2);
                break
                
    return A_Mat_img, A_Mat

## Returns polymer Major axis endpoint and the midpoints
def majorAxisPoints(poly):
    temp = np.zeros([3,2])
    ang = poly[2]*np.pi/180
    
    ## Major axis end point 1 
    temp[0,0] = int(poly[1] - poly[3]*np.cos(ang)) 
    temp[0,1] = int(poly[0] - poly[3]*np.sin(ang))
    
    ## centroid
    temp[1,0] = poly[1]
    temp[1,1] = poly[0]
    
    ## Major axis end point 2
    temp[2,0] = int(poly[1] + poly[3]*np.cos(ang))
    temp[2,1] = int(poly[0] + poly[3]*np.sin(ang))
    
    return temp

## Return minimum distance b/w two sets of points
def minDist(pts1, pts2):
    minD = np.linalg.norm(pts1[0]-pts2[0]) 
    
    for i in range(len(pts1)):
        for j in range(len(pts2)):
            temp = np.linalg.norm(pts1[i]-pts2[j])
            if temp < minD:
                minD = temp
    return minD

## Connected Components:
def ConnecComp(img, A_Mat, props, c_size):
    polyProps = props 
    N = A_Mat.shape[0]
    visited = np.zeros(N)
    cc = []
    
    for v in range(N):
        if visited[v] == 0:
            temp = []
            cc.append(DFSUtil(temp, v, visited, N, A_Mat))
    
    ccImg = np.copy(img)
    startAngle = 0
    endAngle = 360
    color = (255, 0, 0)
    thickness = 2

    AllClusterPointCloud = []
    crystalAngles = []
    for i in range(len(cc)):
        if len(cc[i]) >= c_size:
            majorsAxisPointCloud = []
            ellipseAngels = []
            for j in cc[i]:
                poly = polyProps[j]
                ccImg = cv2.ellipse(ccImg, (int(poly[1]), int(poly[0])), (int(poly[3]), int(poly[4])), poly[2], 0.0, 360.0, (255, 0, 0), 2);
                temp = majorAxisPoints(poly)
                #ccImg = cv2.circle(ccImg, (int(temp[0,0]),int(temp[0,1])), radius=5, color=(255, 0, 0), thickness=-1)
                #ccImg = cv2.circle(ccImg, (int(temp[1,0]),int(temp[1,1])), radius=5, color=(255, 0, 0), thickness=-1)
                temp[temp<0] = 0
                temp[temp>=img.shape[0]] = img.shape[0]-1
                temp = temp.astype('int32')
                
                majorsAxisPointCloud.append(temp[0,:])
                majorsAxisPointCloud.append(temp[2,:])
                ellipseAngels.append(poly[2])
            AllClusterPointCloud.append(majorsAxisPointCloud)
            crystalAngles.append(mean(ellipseAngels))
    
    return ccImg, AllClusterPointCloud, crystalAngles

def PlottingAndSaving(img, ClusterPointCloud, ProjectPath, ResultDir, ImgName, crystalAng, ResultDisp):
    RGBImg = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    CrystalImg = img#invertBinaryImage(finalImg)
    arrLen = 200
    figure, axes = plt.subplots(nrows=1, ncols=2, figsize = (50,25))
    axes[1].imshow(RGBImg, vmin=0, vmax=255)

    crystalArea = []
    centroid = []
    leftTop = []
    bottomRight = []
    boundingBox = []
    for ind, cluster in enumerate(ClusterPointCloud):
        PointCloud = np.array(cluster)
        hull = ConvexHull(PointCloud)
        crystalArea.append(hull.area)
        #Get centoid
        cx = np.mean(hull.points[hull.vertices,0])
        cy = np.mean(hull.points[hull.vertices,1])
        centroid.append([cx, cy])
        
        # Get Boundingbox coordinates
        x_minMax = [np.amin(hull.points[hull.vertices,0]), np.amax(hull.points[hull.vertices,0])]
        y_minMax = [np.amin(hull.points[hull.vertices,1]), np.amax(hull.points[hull.vertices,1])]
        leftTop.append([x_minMax[0], y_minMax[0]])
        bottomRight.append([x_minMax[1],y_minMax[1]])
        #boundingBox.append(y_minMax)
        color = ['b', 'g', 'r', 'c', 'm','y','w']
        c= random.choice(color)
        plt.arrow(cx, cy,arrLen*np.cos(crystalAng[ind]*np.pi/180), arrLen*np.sin(crystalAng[ind]*np.pi/180),linewidth=7.0, color=c )
        for simplex in hull.simplices:
            plt.plot(PointCloud[simplex, 0], PointCloud[simplex, 1], linewidth = 7.0, color=c)

    boundingBox.append(leftTop)
    boundingBox.append(bottomRight)
    axes[0].imshow(img, cmap = 'gray')
    figure.tight_layout()
    figure.savefig(ProjectPath + ResultDir + ImgName[:-4]+'.png')
    if ResultDisp == 1:
        plt.show()
    figure.clf()
    plt.close()
    plt.clf()
    gc.collect()
    return crystalArea, centroid, boundingBox

def ConcatAndShow(input, border, output, scale_percent,text):
    img=np.concatenate((input,border,output),axis=1)
    print(text + " \n")
    show_scalled_img(img,scale_percent)

In [36]:
def GRATE(projectPath, dataDir, imgName, resultDir, annotationDir, dspace_nm,pix2nm):
    img = cv2.imread(projectPath + dataDir + imgName,0)
    if img.any() == None:
        print("IMAGE NOT LOADED")
#     #################### Temporary part ######################################
#     x_c = 1660
#     y_c = 1218
#     boxSize = 1000
#     img = img[y_c-boxSize: y_c+boxSize, x_c-boxSize:x_c+boxSize]
#     ##########################################################################
    
    dspace_pix = int(dspace_nm*pix2nm)
    print("dspace_pix:", dspace_pix)
    image_scale_percent = 50 # Scaling the image before display
    border = np.zeros((img.shape[0],75)) # Setting border between the concatenated images
    
    
    blur_thresh_interation = 200 # prev = 200, 150
    dspace_frac_Blur_kernel = 0.09 # Best till now 0.09, Fraction of dspace for the blur kernel size 
    closing_k_size = 15 # Kernel Size
    opening_k_size = 17 # Kernel Size
    pixThresh = int(1.25*dspace_pix) # 1.25*dspace_pix,Threshold number of pixels consituting polymers
    ellipse_len = int(1.5*dspace_pix) # Old = 160,Breaking polymer into this size before constructing ellipse 
    ellipseAspectRatio = 5 # Threshold aspect Ratio of the ellipse
    thresh_dist = int(1.35*dspace_pix)  # Old = 250, Distance threshold for adjacency matrix 
    thresh_theta = 10  # delta Theta threshold for adjacency matrix 
    clusterSize = 10 # old = 10, Threshold Crystal cluster size 
    
    debug = 0 # To print images: 1, Not to:0
    saveImg = 0 # To intermediate step images: 1, Not to:0
    ResultDisp = 0 # To display final result in notebook: 1, Not to:0 
    
    thresh = BlurThresh(img, blur_thresh_interation, dspace_pix, dspace_frac_Blur_kernel)
    if debug == 1:
        ConcatAndShow(img, border, thresh, image_scale_percent, "BLURRING AND THRESHOLDING")
    if saveImg == 1:
        cv2.imwrite("BLURRING_AND_THRESHOLDING.png", thresh)
    
    closing = Closing(thresh, closing_k_size)
    if debug == 1:
        ConcatAndShow(thresh, border, closing, image_scale_percent, "CLOSING")
    if saveImg == 1:
        cv2.imwrite("CLOSING.png", closing)
        
    opening = Opening(closing, opening_k_size)
    if debug == 1:
        ConcatAndShow(closing, border, opening, image_scale_percent, "OPENING")
    if saveImg == 1:
        cv2.imwrite("OPENING.png", opening)
        
    skeleton = Skeletonize(opening)
    if debug == 1:
        ConcatAndShow(opening, border, skeleton, image_scale_percent, "SKELETONIZED")
    if saveImg == 1:
        Invskeleton = invertBinaryImage(skeleton)
        cv2.imwrite("SKELETONIZED.png", Invskeleton)
        
    skeleton = BreakBraches(skeleton)
    if debug == 1:
        print("BRANCHED SKELETON \n")
        show_scalled_img(skeleton, image_scale_percent)
    if saveImg == 1:
        Invskeleton = invertBinaryImage(skeleton)
        cv2.imwrite("BRANCHED_SKELETON.png", Invskeleton)
        
    _, temp = SkeletonSegmentation(skeleton)
    Broken_backbone_img = Filtered_Uniform_BB(img, temp, pixThresh, ellipse_len)
    if debug == 1:
        print("FILTERED AND UNIFORM SIZED BACKBONE \n")
        show_scalled_img(Broken_backbone_img, image_scale_percent)
    if saveImg == 1:
        InvBroken_backbone_img = invertBinaryImage(Broken_backbone_img)
        cv2.imwrite("FILTERED_AND_UNIFORM_SIZED_BACKBONE.png", InvBroken_backbone_img)
        
    ######################### MODIFY Filtered_Uniform_BB TO REMOVE RESIDUAL ALSO ################################
#     _, filteredPolys = SkeletonSegmentation(Broken_backbone_img_temp)
#     Broken_backbone_img = Remove_small_BB(Broken_backbone_img_temp, filteredPolys, int(0.5*ellipse_len))
    
#     _, filteredPolys1 = SkeletonSegmentation(Broken_backbone_img)
    
#     restructuredFP = RemovingDim(filteredPolys1)
    
    bb_ellipse1, bb_ellipse_props = EllipseConstruction(Broken_backbone_img, ellipseAspectRatio)
    if debug == 1:
        print("ELLIPSE INSCRIBED \n")
        show_scalled_img(bb_ellipse1,image_scale_percent)
    if saveImg == 1:
        Invbb_ellipse1 = invertBinaryImage(bb_ellipse1)
        cv2.imwrite("ELLIPSE_INSCRIBED.png", Invbb_ellipse1)
        
    adjEllipse_img, adjacencyMat = AdjacencyMat(Broken_backbone_img, bb_ellipse_props, thresh_dist, thresh_theta)
    if debug == 1:
        print("ADJACENT ELLIPSES \n")
        show_scalled_img(adjEllipse_img, image_scale_percent)
    if saveImg == 1:
        InvadjEllipse_img = invertBinaryImage(adjEllipse_img)
        cv2.imwrite("ADJACENT_ELLIPSES.png", InvadjEllipse_img)
        
    ellipseCluster, AllClusterPointCloud, crystalAngles = ConnecComp(Broken_backbone_img, adjacencyMat, bb_ellipse_props, clusterSize)
    if debug == 1:
        print("CLUSTERS")
        show_scalled_img(ellipseCluster, image_scale_percent)
    if saveImg == 1:
        InvellipseCluster = invertBinaryImage(ellipseCluster)
        cv2.imwrite("CLUSTERS.png", InvellipseCluster)
        
    crystalArea, centroid, boundBox = PlottingAndSaving(img, AllClusterPointCloud, projectPath, resultDir, imgName, crystalAngles, ResultDisp)
    #print(boundBox)
    df = pd.DataFrame(list(zip(centroid, crystalArea, crystalAngles)), columns=["Centroid", 'Crystal Area (pixel^2)', 'Crystal Angle (zero at X-axis and clockwise positive)'])
    df = df.round(2)

    df_annotation = pd.DataFrame(list(zip(boundBox[0], boundBox[1])), columns=["left Top", "Bottom Right"])
    print(projectPath + resultDir +imgName[:-4]+'.csv')
    df.to_csv(projectPath + resultDir +imgName[:-4]+'.csv')
    df_annotation.to_csv(projectPath + annotationDir + imgName[:-4]+'.csv')
    
    return boundBox

## GRATE as Function

In [37]:
from os import listdir
from os.path import isfile, join
projectPath = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/'
dataDir = 'sampleData/'
resultDir = 'Results/'
AnnotationDir = 'Annotations/'
# onlyfiles = [f for f in listdir(projectPath+dataDir) if isfile(join(projectPath+dataDir, f))]
file = open(projectPath + 'failed.txt', 'r')
dspace_nm = 1.9
pix2nm = 78.5

for f in file.readlines():
    f = f[:-1]
    #if f == "FoilHole_21830409_Data_21829764_21829765_20200122_1247.tif":
    print(projectPath+dataDir+f)    
    bb =GRATE(projectPath, dataDir, f, resultDir, AnnotationDir, dspace_nm,pix2nm)

/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21830409_Data_21829764_21829765_20200122_1247.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830409_Data_21829764_21829765_20200122_1247.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21830304_Data_21829764_21829765_20200122_1151.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830304_Data_21829764_21829765_20200122_1151.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21830342_Data_21829764_21829765_20200122_1215.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830342_Data_21829764_21829765_20200122_1215.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21836266_Data_21829764_21829765_20200123_0033.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Pr

/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830324_Data_21829764_21829765_20200122_1205.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21836371_Data_21829764_21829765_20200123_0114.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21836371_Data_21829764_21829765_20200123_0114.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21836262_Data_21829764_21829765_20200123_0029.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21836262_Data_21829764_21829765_20200123_0029.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21832455_Data_21829764_21829765_20200122_1343.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21832455_Data_21829764_21829765_20200122_1343.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_

/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21836357_Data_21829764_21829765_20200123_0105.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21832595_Data_21829764_21829765_20200122_1518.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21832595_Data_21829764_21829765_20200122_1518.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21832488_Data_21829764_21829765_20200122_1410.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21832488_Data_21829764_21829765_20200122_1410.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21830465_Data_21829764_21829765_20200122_1308.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830465_Data_21829764_21829765_20200122_1308.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_

/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830340_Data_21829764_21829765_20200122_1214.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21830529_Data_21829764_21829765_20200122_1325.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21830529_Data_21829764_21829765_20200122_1325.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21832454_Data_21829764_21829765_20200122_1343.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21832454_Data_21829764_21829765_20200122_1343.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21832506_Data_21829764_21829765_20200122_1423.tif
dspace_pix: 149
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/Results/FoilHole_21832506_Data_21829764_21829765_20200122_1423.csv
/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_

<Figure size 432x288 with 0 Axes>

## END

## 4K to 2K

In [30]:
def breakimgTo4(img):
    img00 = img[0:2048,0:2048]
    img01 = img[0:2048, 2048:]
    img10 = img[2048:, 0:2048]
    img11 = img[2048:, 2048:]
    return img00, img01, img10, img11

def changeToYolo(x_min, x_max, y_min, y_max):
    x = ((x_min + x_max) / 2) / 2048.0
    y = ((y_min + y_max) / 2) / 2048.0
    w = (x_max - x_min) / 2048.0
    h = (y_max - y_min) / 2048.0
    
    return x,y,w,h
    
def breakBB(x_min, x_max, y_min, y_max, howTo):
    
    if howTo == 'h':
        up = [x_min, x_max, y_min, -1]
        dn = [x_min, x_max, 0, y_max]
        return up, dn
    
    if howTo == 'v':
        l = [x_min, -1, y_min, y_max]
        r = [0, x_max, y_min, y_max]
        return l, r
    
    if howTo == 'c':
        q1 = [x_min, -1, y_min, -1]
        q2 = [0, x_max, y_min, -1]
        q3 = [x_min, -1, 0, y_max]
        q4 = [0, x_max, 0, y_max]
        return q1, q2, q3,q4

def shiftAxisEnd(x_min, x_max, y_min, y_max, quad):
    if quad == 1:
        x_min += 2048
        x_max += 2048
        y_min += 2048
        y_max += 2048

    elif quad == 2:
        y_min += 2048
        y_max += 2048
    
    elif quad == 3:
        x_min += 2048
        x_max += 2048
    
    return x_min, x_max, y_min, y_max      

In [39]:
import os
from os import listdir
from os.path import isfile, join
# os.makedirs('dataset/images/train', exist_ok = True)
# os.makedirs('dataset/images/test', exist_ok = True)
# os.makedirs('dataset/labels/train', exist_ok = True)
# os.makedirs('dataset/labels/test', exist_ok = True)

src_imgpath = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/ME592x/Data/FullSize/'
dst_imgpath = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/ME592x/Data/2k'
src_labelpath = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/ME592x/Annotations/BB/4k/'
dst_labelpath = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/ME592x/Annotations/BB/2k'
img_res = 4096
files = listdir(src_imgpath)

print(len(files))

# train_files = files[:int(0.75*len(files))]
# test_files = files[int(0.75*len(files)):]
# print(len(train_files))
# print(len(test_files))

for f in files:
    data, lt, rb = [], [], []
    src_label = join(src_labelpath,f.replace('.tif', '.csv'))
#     label_file = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/ME592x/Annotations/BB/2k/%s'%(f.replace('.tif', '.txt'))
    dst_label00 = join(dst_labelpath, f.replace('.tif', '_00.txt'))
    dst_label01 = join(dst_labelpath, f.replace('.tif', '_01.txt'))
    dst_label10 = join(dst_labelpath, f.replace('.tif', '_10.txt'))
    dst_label11 = join(dst_labelpath, f.replace('.tif', '_11.txt'))

    
#     src_img_path = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/ME592x/Data/FullSize/%s'%f
    src_img = join(src_imgpath, f)    
    dst_img00 = join(dst_imgpath,f.replace('.tif', '_00.jpg'))
    dst_img01 = join(dst_imgpath,f.replace('.tif', '_01.jpg'))
    dst_img10 = join(dst_imgpath,f.replace('.tif', '_10.jpg'))
    dst_img11 = join(dst_imgpath,f.replace('.tif', '_11.jpg'))

#     if not os.path.exists(os.path.join('/work/adarsh/Dhruv/GRATE/ME592x/Annotations/BB', csv_file)):
#         shutil.copyfile(src_img_path, dst_img_path)
#         continue

    if os.path.exists(src_label):
        data = pd.read_csv(src_label)
        img = cv2.imread(src_img, 0)
        
        img00, img01, img10, img11 = breakimgTo4(img)
        cv2.imwrite(dst_img00, img00)
        cv2.imwrite(dst_img01, img01)
        cv2.imwrite(dst_img10, img10)
        cv2.imwrite(dst_img11, img11)
        
        if len(data)==0:
            open(dst_label00, 'a').close()
            open(dst_label01, 'a').close()
            open(dst_label10, 'a').close()
            open(dst_label11, 'a').close()
#             shutil.copyfile(src_img_path, dst_img_path)
            continue

        lt = data['left Top'].tolist()
        rb = data['Bottom Right'].tolist()
        md00 = []
        md01 = []
        md10 = []
        md11 = []

        for i in range(len(data)):
            X_min, Y_min = float(lt[i][1:-1].split(',')[0]), float(lt[i][1:-1].split(',')[1])
            X_max, Y_max = float(rb[i][1:-1].split(',')[0]), float(rb[i][1:-1].split(',')[1])
            
            x_min = X_min-int(img_res/2)
            y_min = Y_min-int(img_res/2)
            x_max = X_max-int(img_res/2)
            y_max = Y_max-int(img_res/2)
#             print("original:", x_min, x_max, y_min, y_max)
            if x_min == 0:
                x_min+=1
            if x_max == 0:
                x_max-=1
            if y_min == 0:
                y_min+=1
            if y_max == 0:
                y_max-=1
            
            ##CASE 1: x_min and x_max are negative
            if x_max<0:
                if y_max<0:
                    ## 1st Quad ##
                    x_min, x_max, y_min, y_max = shiftAxisEnd(x_min, x_max, y_min, y_max, 1)
                    x,y,w,h = changeToYolo(x_min, x_max, y_min, y_max)
                    md00.append([int(0), x, y, w, h])
                
                elif y_min>0:
                    ## 3rd Quad ##
                    x_min, x_max, y_min, y_max = shiftAxisEnd(x_min, x_max, y_min, y_max, 3)
                    x,y,w,h = changeToYolo(x_min, x_max, y_min, y_max)
#                     print("1",x,y,w,h)
                    md10.append([int(0), x, y, w, h])
                    
                else:
                    ## 1st and 3rd Quad ##
                    up, dn = breakBB(x_min, x_max, y_min, y_max, 'h')
                    up[0], up[1], up[2], up[3] = shiftAxisEnd(up[0], up[1], up[2], up[3], 1)
                    dn[0], dn[1], dn[2], dn[3] = shiftAxisEnd(dn[0], dn[1], dn[2], dn[3], 3)
                    ux, uy, uw, uh = changeToYolo(up[0], up[1], up[2], up[3])
                    dx, dy, dw, dh = changeToYolo(dn[0], dn[1], dn[2], dn[3])
                    md00.append([int(0), ux, uy, uw, uh])
#                     print("2",dx, dy, dw, dh)
                    md10.append([int(0), dx, dy, dw, dh])
            
            elif x_min>0:
                if y_max<0:
                    ## 2nd Quad ##
                    x_min, x_max, y_min, y_max = shiftAxisEnd(x_min, x_max, y_min, y_max, 2)
                    x,y,w,h = changeToYolo(x_min, x_max, y_min, y_max)
                    md01.append([int(0), x, y, w, h])
                
                elif y_min>0:
                    ## 4th Quad ##
                    x_min, x_max, y_min, y_max = shiftAxisEnd(x_min, x_max, y_min, y_max, 4)
                    x,y,w,h = changeToYolo(x_min, x_max, y_min, y_max)
                    md11.append([int(0), x, y, w, h])
                    
                else:
                    ## 2nd and 4th Quad ##
                    up, dn = breakBB(x_min, x_max, y_min, y_max, 'h')
                    up[0], up[1], up[2], up[3] = shiftAxisEnd(up[0], up[1], up[2], up[3], 2)
                    dn[0], dn[1], dn[2], dn[3] = shiftAxisEnd(dn[0], dn[1], dn[2], dn[3], 4)
                    ux, uy, uw, uh = changeToYolo(up[0], up[1], up[2], up[3])
                    dx, dy, dw, dh = changeToYolo(dn[0], dn[1], dn[2], dn[3])
                    md01.append([int(0), ux, uy, uw, uh])
                    md11.append([int(0), dx, dy, dw, dh])
            
            elif x_min<0 and x_max>0:
                if y_max<0:
                    ## 1st and 2nd Quad ##
                    l,r = breakBB(x_min, x_max, y_min, y_max, 'v')
                    l[0], l[1], l[2], l[3] = shiftAxisEnd(l[0], l[1], l[2], l[3], 1)
                    r[0], r[1], r[2], r[3] = shiftAxisEnd(r[0], r[1], r[2], r[3], 2)
                    lx, ly, lw, lh = changeToYolo(l[0], l[1], l[2], l[3])
                    rx, ry, rw, rh = changeToYolo(r[0], r[1], r[2], r[3])
                    md00.append([int(0), lx, ly, lw, lh])
                    md01.append([int(0), rx, ry, rw, rh])
                
                elif y_min>0:
                    ## 3rd and 4th Quad ##
                    l,r = breakBB(x_min, x_max, y_min, y_max, 'v')
                    l[0], l[1], l[2], l[3] = shiftAxisEnd(l[0], l[1], l[2], l[3], 3)
                    r[0], r[1], r[2], r[3] = shiftAxisEnd(r[0], r[1], r[2], r[3], 4)
                    lx, ly, lw, lh = changeToYolo(l[0], l[1], l[2], l[3])
                    rx, ry, rw, rh = changeToYolo(r[0], r[1], r[2], r[3])
#                     print('3',lx, ly, lw, lh)
                    md10.append([int(0), lx, ly, lw, lh])
                    md11.append([int(0), rx, ry, rw, rh])
                    
                else:
                    ## All 4 Quad ##
#                     print('4 before breaking', x_min, x_max, y_min, y_max)
                    q1, q2, q3, q4 = breakBB(x_min, x_max, y_min, y_max, 'c')
#                     print('4 after Breaking: ', q3)
                    q1[0], q1[1], q1[2], q1[3] = shiftAxisEnd(q1[0], q1[1], q1[2], q1[3],1)
                    q2[0], q2[1], q2[2], q2[3] = shiftAxisEnd(q2[0], q2[1], q2[2], q2[3],2)
                    q3[0], q3[1], q3[2], q3[3] = shiftAxisEnd(q3[0], q3[1], q3[2], q3[3],3)
                    q4[0], q4[1], q4[2], q4[3] = shiftAxisEnd(q4[0], q4[1], q4[2], q4[3],4)
                    q1x, q1y, q1w, q1h = changeToYolo(q1[0], q1[1], q1[2], q1[3])
                    q2x, q2y, q2w, q2h = changeToYolo(q2[0], q2[1], q2[2], q2[3])
                    q3x, q3y, q3w, q3h = changeToYolo(q3[0], q3[1], q3[2], q3[3])
                    q4x, q4y, q4w, q4h = changeToYolo(q4[0], q4[1], q4[2], q4[3])
                    md00.append([int(0), q1x, q1y, q1w, q1h])
                    md01.append([int(0), q2x, q2y, q2w, q2h])
#                     print('4',q3x, q3y, q3w, q3h)
                    md10.append([int(0), q3x, q3y, q3w, q3h])
                    md11.append([int(0), q4x, q4y, q4w, q4h])
            
        
        if not md00:
            open(dst_label00, 'a').close()
        else:
            np.savetxt(os.path.join(dst_label00), md00,fmt=['%d', '%f', '%f', '%f', '%f'] )
             
        if not md01:  
            open(dst_label01, 'a').close()
        else:
            np.savetxt(os.path.join(dst_label01), md01,fmt=['%d', '%f', '%f', '%f', '%f'] )
            
        if not md10:
            open(dst_label10, 'a').close()
        else:
            np.savetxt(os.path.join(dst_label10), md10,fmt=['%d', '%f', '%f', '%f', '%f'] )
             
        if not md11:
            open(dst_label11, 'a').close()
        else:
            np.savetxt(os.path.join(dst_label11), md11,fmt=['%d', '%f', '%f', '%f', '%f'] )

637


In [9]:
from os import listdir
from os.path import isfile, join
projectPath = '/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/'
dataDir = 'sampleData/'
resultDir = 'Results/'
AnnotationDir = 'Annotations/'
# onlyfiles = [f for f in listdir(projectPath+dataDir) if isfile(join(projectPath+dataDir, f))]
file = open(projectPath + 'failed.txt', 'r')
dspace_nm = 1.9
pix2nm = 78.5

for f in file.readlines():
    f = f[:-1]
    #if f == "FoilHole_21830409_Data_21829764_21829765_20200122_1247.tif":
    print(projectPath+dataDir+f)    
    bb =GRATE(projectPath, dataDir, f, resultDir, AnnotationDir, dspace_nm,pix2nm)

/home/dhruv/Dhruv/ISU/PhD/Projects/GRATE/GRATE_for_PennState/sampleData/FoilHole_21830409_Data_21829764_21829765_20200122_1247.tif


NameError: name 'GRATE' is not defined